# Tổng quan phần thu thập dữ liệu
1. Mục tiêu
2. Công cụ sử dụng
3. Các bước thu thập dữ liệu

In [123]:
# import những  thư viện cần thiết
import pandas as pd 
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Hàm gửi yêu cầu đến server yêu cầu tải toàn bộ nội dung trang web

In [127]:

def get_content(url, headers):
    # time.sleep(1)
    session = HTMLSession()
    response = session.get(url, headers=headers)
    if(response.status_code == 200):
        print("Success!!!")
        return response
    return None

In [128]:
BASE_URL = 'https://vieclam24h.vn'


headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}


## Step 1: Lấy những liên kết của trang web công việc
- Đầu tiên, khi vô một trang web tìm việc, ta sẽ thấy một loạt danh sách các công việc, sau khi nhấn vào từng cái sẽ liên kết trực tiếp đến trang chi tiết công việc. Vậy đầu tiên ta đến trang tìm kiếm và lọc tất cả các đường dẫn đến công việc.

In [129]:
# Code demo request lấy page list công việc
job_pool = pd.DataFrame()

for i in range(100,101):
   # Khởi tạo url vầ yêu cầu lấy page về 
   url = BASE_URL + '/tim-kiem-viec-lam-nhanh?page={}'.format(i)
   
   print(url)
   
   page_content = get_content(url, headers)
   
   soup = BeautifulSoup(page_content.text, 'html.parser')
   
   containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
   print('Đã tìm thấy {} công việc'.format(len(containers)))
   for container in containers:
      dict_job = {}
      
      buffer = BASE_URL + container.find('a', href=True)['href']
      dict_job['Liên kết'] = buffer if buffer else None
      
      buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
      dict_job['Tên công việc'] = buffer.text if buffer else None
      
      buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
      dict_job['Tên công ty'] = buffer.text if buffer else None
      
      buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
      dict_job['Mức lương'] = buffer.text if buffer else None
      
      buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
      dict_job['Khu vực tuyển'] = buffer.text if buffer else None
      
      job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool.sample(5)
         


https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=100
Success!!!
Đã tìm thấy 30 công việc


,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
9,https://vieclam24h.vn/hanh-chinh-thu-ky/cham-s...,Chăm Sóc Khách Hàng (Đi Làm Ngay - Không Yêu C...,Công Ty Cổ Phần Bất Động Sản Cona Land,8 - 10 triệu,TP.HCM
29,https://vieclam24h.vn/ban-hang-kinh-doanh/chuy...,Chuyên Viên Kinh Doanh Dịch Vụ Viễn Thông Fpt ...,Chi Nhánh Công Ty Cổ Phần Viễn Thông Fpt,8 - 20 triệu,TP.HCM
22,https://vieclam24h.vn/ban-hang-kinh-doanh/tap-...,"Tập Sự Đại Diện Bán Hàng, Kênh GT",Công Ty Cổ Phần Mondelez Kinh Đô Việt Nam,10 - 15 triệu,"Hà Nội, Hải Phòng, Quảng Bình, Ninh Bình, +5"
19,https://vieclam24h.vn/ban-si-ban-le-quan-ly-cu...,Nabati - Nhân Viên Bán Hàng Thị Trường - Khu V...,Công Ty TNHH Nabati Việt Nam,9 - 13 triệu,"Vĩnh Long, Cần Thơ"
3,https://vieclam24h.vn/marketing/nhan-vien-kinh...,Nhân Viên Kinh Doanh,Công Ty TNHH Sản Xuất Thịnh Việt,20 - 35 triệu,Bình Dương


In [130]:
# Lưu dữ liệu vào file csv
job_pool.to_csv('../data/job_pool.csv', index=False)

# Step 2: Thu thập dữ liệu của từng trang web
- Sau khi thu thập được các link trang web, ta tiến hành thu thập dữ liệu của từng trang web bằng cách sử dụng cách tương tự

In [131]:
# Đọc dữ liệu từ file job_links.csv
df = pd.read_csv('../data/job_pool.csv')
df.sample(5)

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
8,https://vieclam24h.vn/khoa-hoc-ky-thuat/nhan-v...,Nhân Viên Kỹ Thuật Thi Công Điện - Điện Nhẹ Tạ...,Công Ty Cổ Phần Bằng Hữu Kinh Doanh,8 - 15 triệu,TP.HCM
22,https://vieclam24h.vn/ban-hang-kinh-doanh/tap-...,"Tập Sự Đại Diện Bán Hàng, Kênh GT",Công Ty Cổ Phần Mondelez Kinh Đô Việt Nam,10 - 15 triệu,"Hà Nội, Hải Phòng, Quảng Bình, Ninh Bình, +5"
16,https://vieclam24h.vn/ke-toan/nhan-vien-ke-toa...,Nhân Viên Kế Toán Tổng Hợp,Công Ty TNHH Thương Mại Dịch Vụ Green Leaf Việ...,12 - 15 triệu,TP.HCM
2,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Nhân Viên Phụ Bếp Nam,Công Ty Cổ Phần City Kitchen,4 - 10 triệu,TP.HCM
3,https://vieclam24h.vn/marketing/nhan-vien-kinh...,Nhân Viên Kinh Doanh,Công Ty TNHH Sản Xuất Thịnh Việt,20 - 35 triệu,Bình Dương


In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Liên kết       30 non-null     object
 1   Tên công việc  30 non-null     object
 2   Tên công ty    30 non-null     object
 3   Mức lương      30 non-null     object
 4   Khu vực tuyển  30 non-null     object
dtypes: object(5)
memory usage: 1.3+ KB


In [138]:
craw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển', 
# Thông tin chung    
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])
for url in df['Liên kết']:

    job_content = get_content(url, headers=headers).text
    soup = BeautifulSoup(job_content, 'html.parser')

    dict_job = {}

    dict_job['Liên kết'] = url

    buffer = soup.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
    dict_job['Tên công ty'] = buffer.text if buffer else None

    buffer = soup.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
    dict_job['Tên công việc'] = buffer.text if buffer else None

    buffer = soup.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
    dict_job['Mức lương'] = buffer.text if buffer else None

    buffer = soup.find('a', class_='hover:text-se-accent')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None


    buffer = soup.find('p', string="Cấp bậc")
    dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Số lượng tuyển")
    dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Hình thức làm việc")
    dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Yêu cầu bằng cấp")
    dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
    dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Ngành nghề")
    dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


    buffer1 = soup.find('h4', string="Mô tả công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Yêu cầu công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Quyền lợi")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Địa điểm làm việc")
    buffer2 = buffer1.find_next_sibling('ul')
    dict_job['Địa điểm làm việc'] = buffer2.text if buffer2 else None 


    buffer1 = soup.find('h3', string="Từ khoá")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Từ khóa'] = buffer2.text if buffer2 else None

    company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

    buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
    for buffer in buffers:
        if buffer.get_text() == 'Địa chỉ:':
            dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
        if buffer.get_text() == 'Quy mô:':
            dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

    craw_job_details = pd.concat([craw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!


In [139]:
craw_job_details.sample(5)

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
13,https://vieclam24h.vn/marketing/nhan-vien-mark...,Công Ty Tnhh Kim Cương Xanh Bà Rịa,Nhân Viên Marketing Excutive (Lương Từ 15 - 20...,15 - 20 triệu,Bà Rịa - Vũng Tàu,Chuyên viên- nhân viên,2,Toàn thời gian cố định,Cao đẳng,2 năm,Marketing/Hành chính - Thư ký,"Bà Rịa - Vũng Tàu, 398 Cách mạng tháng 8 Phước...",1. Nội dung Marketing (Branding & Social)• Xây...,"• Có tối thiểu 2 năm kinh nghiệm marketing, ưu...",• Mức lương: thoả thuận ( 15 - 20 triệu).• Môi...,lươngthị xã bà rịathành phố vũng tàuViệc làm B...,"398 Cách Mạng Tháng 8, Phường Phước Trung, Thà...",10 - 150 người
22,https://vieclam24h.vn/ban-hang-kinh-doanh/tap-...,Công Ty Cổ Phần Mondelez Kinh Đô Việt Nam,"Tập Sự Đại Diện Bán Hàng, Kênh GT",10 - 15 triệu,"Hà Nội,",Chuyên viên- nhân viên,20,Toàn thời gian cố định,None,Chưa có kinh nghiệm,Bán hàng - Kinh doanh/Lao động phổ thông/Bán s...,"Hà Nội, Khu vực Hà Nội, Thanh XuânHải Phòng, T...","Khu vực:Miền Nam: Hồ Chí Minh, Bình Dương, Lon...",- KHÔNG YÊU CẦU KINH NGHIỆM- Tốt nghiệp từ THP...,- Được đào tạo bài bản về kỹ năng bán hàng- Có...,bán hàngkênh gtnhân viên bán hàng tại đà nẵngV...,"138 – 142 Hai Ba Trung St., Dakao Ward, Dist. ...",Trên 300 người
26,https://vieclam24h.vn/khoa-hoc-ky-thuat/nhan-v...,Công Ty TNHH Chuyển Giao Công Nghệ Và Dịch Vụ ...,Nhân Viên Cơ Khí - Lương Đến 10 Triệu,7 - 10 triệu,Hà Nội,Chuyên viên- nhân viên,2,Toàn thời gian cố định,Trung cấp,Chưa có kinh nghiệm,Khoa học - Kỹ thuật/Điện - Điện tử - Điện lạnh...,"Hà Nội, Tây Hồ",- Vận hành máy phay CNC./ hoặc tiện CNC.- Lắp ...,"- Tốt nghiệp trung cấp ,cao đẳng chuyên ngành ...",- Tăng lương theo năng lực. Có thể thỏa thuận ...,lươnghuyện an dươngnhân viên cViệc làm Hà NộiK...,"Số nhà 26 ngõ 41 Đường An Dương Vương, phường ...",10 - 150 người
15,https://vieclam24h.vn/xay-dung/can-bo-ky-thuat...,Công ty CP đá Granite & Marble Tự Nhiên Thiên Sơn,Cán Bộ Kỹ Thuật - Giám Sát Hiện Trường Dự Án (...,15 - 22 triệu,Hà Nội,Chuyên viên- nhân viên,3,Toàn thời gian cố định,Cao đẳng,3 năm,Xây dựng/Khoa học - Kỹ thuật/Kiến trúc - Thiết...,"Hà Nội, KĐT Nam Trung Yên - Cầu Giấy - Hà Nội,...",- Quản lý công trình tại các dự án thi công ốp...,- Tuổi tuyển dụng: 24 - 35 tuổi. Trình độ Cao ...,- Thử việc: 14.000.000 – 20.000.000 đồng / thá...,nhậphiện trườnggiám sát hiện trườngViệc làm Hà...,"Số 85 D6, Khu ĐTM Đại Kim, P.Đại Kim, Q.Hoàng ...",150 - 300 người
21,https://vieclam24h.vn/ke-toan/nhan-vien-ke-toa...,Công Ty TNHH Thương Mại Dịch Vụ Sg Law,Nhân Viên Kế Toán Quận 10 - Đi Làm Ngay,8 - 15 triệu,TP.HCM,Chuyên viên- nhân viên,3,Toàn thời gian cố định,Đại học,1 năm,Kế toán/Kiểm toán/Hành chính - Thư ký,"TP.HCM, 618 Đường 3/2, Phường 14, Quận 10TP.HC...",Theo dõi và quản lý công nợ toàn Công ty.Xác đ...,Trình độ:Tốt nghiệp Đại học chuyên ngành Kế to...,Lương thỏa thuận theo năng lựcHưởng đầy đủ chế...,kế toánkế toán quận 10toánViệc làm TP.HCMKế to...,"618 Đường 3/2, Phường 14, Quận 10, Thành phố H...",10 - 150 người


In [140]:
# Lưu vào file craw_dataset.csv
craw_job_details.info()
craw_job_details.to_csv('../data/craw_dataset.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Liên kết             30 non-null     object
 1   Tên công ty          30 non-null     object
 2   Tên công việc        30 non-null     object
 3   Mức lương            30 non-null     object
 4   Khu vực tuyển        30 non-null     object
 5   Cấp bậc              30 non-null     object
 6   Số lượng tuyển       30 non-null     object
 7   Hình thức làm việc   30 non-null     object
 8   Yêu cầu bằng cấp     25 non-null     object
 9   Yêu cầu kinh nghiệm  30 non-null     object
 10  Ngành nghề           30 non-null     object
 11  Địa điểm làm việc    30 non-null     object
 12  Mô tả công việc      30 non-null     object
 13  Yêu cầu công việc    30 non-null     object
 14  Quyền lợi            30 non-null     object
 15  Từ khóa              30 non-null     object
 16  Địa chỉ cô

# Một số phần demo để testing
- Đây là đoạn code để người collect data có thể test thử trước khi chạy toàn bộ code

In [27]:
# Demo lất page content
with open('../data/page_content.txt', 'r', encoding='utf-8') as f:
    page_text = f.read()

job_pool = pd.DataFrame(columns=['Liên kết', 'Tên công việc','Tên công ty','Mức lương','Khu vực tuyển'])

soup = BeautifulSoup(page_text, 'html.parser')
containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
print('Đã tìm thấy {} công việc'.format(len(containers)))
for container in containers:
    dict_job = {}
    
    buffer = BASE_URL + container.find('a', href=True)['href']
    dict_job['Liên kết'] = buffer if buffer else None
    
    buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
    dict_job['Tên công việc'] = buffer.text if buffer else None
    
    buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
    dict_job['Tên công ty'] = buffer.text if buffer else None
    
    buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
    dict_job['Mức lương'] = buffer.text if buffer else None
    
    buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None
    
    job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool

Đã tìm thấy 30 công việc


,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
0,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Nhân Viên CSKH - Ứng Dụng Du Lịch Tiếng Anh (Q...,Công Ty TNHH Vietnam Concentrix Services,10 - 15 triệu,TP.HCM
1,https://vieclam24h.vn/hanh-chinh-thu-ky/bien-p...,Biên Phiên Dịch Tiếng Trung Tại Long An,Công Ty Cổ Phần Gia Phú,10 - 20 triệu,Long An
2,https://vieclam24h.vn/ban-hang-kinh-doanh/tro-...,Trợ Lý Phát Triển Kinh Doanh,Công Ty Cổ Phần Giấy CP,12 - 14 triệu,TP.HCM
3,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Telesales (Chợ Tốt Xe),Công Ty TNHH Chợ Tốt Trading,6 - 10 triệu,Hà Nội
4,https://vieclam24h.vn/cham-soc-khach-hang/nhan...,Nhân Viên Dịch Vụ Khách Hàng,Mcredit - Công Ty Tài Chính TNHH Mb Shinsei,10 - 20 triệu,Hà Nội
5,https://vieclam24h.vn/ban-hang-kinh-doanh/chuy...,Chuyên Viên Tư Vấn Tài Chính Hồ Chí Minh (Lươn...,Công Ty TNHH Bảo Hiểm Nhân Thọ AIA Việt Nam,16 - 30 triệu,TP.HCM
6,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Kinh Doanh Nội Thất - Thu Nhập 20-30...,Công Ty TNHH Kiến Trúc Milan,20 - 30 triệu,Hà Nội
7,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Trợ Lý (Biết Tiếng Trung),Công Ty TNHH Việt Nam Paiho,7 - 15 triệu,TP.HCM
8,https://vieclam24h.vn/ke-toan/ke-toan-truong-c...,Kế Toán Trưởng,Công Ty TNHH Thương Mại Dịch Vụ Phát Triển Sản...,18 - 30 triệu,TP.HCM
9,https://vieclam24h.vn/ngan-hang/hcm-giam-doc-q...,HCM - Giám Đốc Quan Hệ Khách Hàng Ưu Tiên,Acb - Ngân Hàng Thương Mại Cổ Phần Á Châu,16 - 50 triệu,TP.HCM


In [32]:

# job = get_content('https://vieclam24h.vn/hanh-chinh-thu-ky/bien-phien-dich-tieng-trung-tai-long-an-c1p123id200290693.html',headers= headers)

# with open('../data/job_content.txt', 'w', encoding='utf-8') as f:
#     f.write(str(job.text))

Success!!!


In [104]:
# Demo lấy chi tiết

craw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển', 
# Thông tin chung    
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

# with open('../data/job_content.txt', 'r', encoding='utf-8') as f:
#     job_content = f.read()
url = 'https://vieclam24h.vn/xay-dung/chi-huy-truong-cong-trinh-c31p73id200288561.html'
job_content = get_content(url, headers=headers).text
soup = BeautifulSoup(job_content, 'html.parser')

dict_job = {}

dict_job['Liên kết'] = 'https://vieclam24h.vn/ban-hang-kinh-doanh/'

buffer = soup.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
dict_job['Tên công ty'] = buffer.text if buffer else None

buffer = soup.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
dict_job['Tên công việc'] = buffer.text if buffer else None

buffer = soup.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
dict_job['Mức lương'] = buffer.text if buffer else None

buffer = soup.find('a', class_='hover:text-se-accent')
dict_job['Khu vực tuyển'] = buffer.text if buffer else None


buffer = soup.find('p', text="Cấp bậc")
dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Số lượng tuyển")
dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Hình thức làm việc")
dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Yêu cầu bằng cấp")
dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Yêu cầu kinh nghiệm")
dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Ngành nghề")
dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


buffer1 = soup.find('h4', text="Mô tả công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Yêu cầu công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Quyền lợi")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Địa điểm làm việc")
buffer2 = buffer1.find_next_sibling('ul')
dict_job['Địa điểm làm việc'] = buffer2.text if buffer2 else None 


buffer1 = soup.find('h3', text="Từ khoá")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Từ khóa'] = buffer2.text if buffer2 else None

company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
for buffer in buffers:
    if buffer.get_text() == 'Địa chỉ:':
        dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
    if buffer.get_text() == 'Quy mô:':
        dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

craw_job_details = pd.concat([craw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
craw_job_details


Success!!!


C:\Users\huynh\AppData\Local\Temp\ipykernel_14372\2993126334.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Cấp bậc")
C:\Users\huynh\AppData\Local\Temp\ipykernel_14372\2993126334.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Số lượng tuyển")
C:\Users\huynh\AppData\Local\Temp\ipykernel_14372\2993126334.py:39: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Hình thức làm việc")
C:\Users\huynh\AppData\Local\Temp\ipykernel_14372\2993126334.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Yêu cầu bằng cấp")
C:\Users\huynh\AppData\Local\Temp\ipykernel_14372\2993126334.py:45: DeprecationWarning: The 'text' argument to find()-type methods is

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
0,https://vieclam24h.vn/ban-hang-kinh-doanh/,Công Ty CP Hóa Chất Xây Dựng Bách Khoa,Chỉ Huy Trưởng Công Trình,13 - 17 triệu,Hà Nội,Quản lý cấp trung,2,Toàn thời gian cố định,Đại học,2 năm,Xây dựng/Quản lý dự án,"Hà Nội, Phòng 2904, tháp A, chung cư thăng lon...",- Điều phối chung các công việc tại dự án.- Là...,- Tốt nghiệp đại học- Có chứng chỉ giám sát hi...,- Lương: Cạnh tranh theo năng lực (Thu nhập : ...,công trìnhchỉ huy trưởng công trình tại hà nội...,"Phòng 2904, tháp A, chung cư thăng long Number...",10 - 150 người
